In [1]:
import os
from dotenv import load_dotenv
import settrade.openapi
from settrade.openapi import Investor
import pandas as pd
import json
import time
import pandas_ta as ta
from datetime import datetime
load_dotenv()

True

In [2]:
GLOBLEX_ID = os.getenv('GLOBLEX_ID')
SETTRADE_APP_ID = os.getenv('SETTRADE_APP_ID')
SETTRADE_APP_SECRET = os.getenv('SETTRADE_APP_SECRET')
# GLOBLEX_ID, SETTRADE_APP_ID, SETTRADE_APP_SECRET

In [3]:
investor = Investor(
    app_id = SETTRADE_APP_ID,
    app_secret = SETTRADE_APP_SECRET,
    broker_id = 'SANDBOX',
    app_code = 'SANDBOX',
    is_auto_queue = False
)

In [4]:
deri = investor.Derivatives(account_no = '980149-0')

In [5]:
data = []

In [6]:
new_df = pd.DataFrame(columns=['time', 'open', 'high', 'low', 'close', 'volumn'])

In [7]:
test_df = pd.DataFrame(columns=['time', 'open', 'high', 'low', 'close', 'volume'])

In [8]:
market = investor.MarketData()
his = market.get_candlestick('GOU21', '1m', 20, '', '', True)
df = pd.DataFrame(his['data'], columns = ['time', 'open', 'high', 'low', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='s')

In [9]:
mqtt = investor.MQTTWebsocket()

In [10]:
def strategy(df):
    df['ema'] = ta.ema(df['close'], length=5)
    df['sma'] = ta.sma(df['close'], length=5)
    df['signal'] = df['ema'] > df['sma']

In [11]:
buy_log = []
sell_log = []

In [17]:
def get_realtime(result, subscriber):
    global his, data, new_df, df, test_df
    
    t = result['data']['time']
    t = pd.to_datetime(t, unit='s')
    
    c = result['data']['close']
    o = result['data']['open']
    h = result['data']['high']
    l = result['data']['low']
    v = result['data']['volume']
    
    data = [t, o, h, l, c, v]
    a = len(new_df.index)
    new_df.loc[a] = data
    
    if len(new_df) > 2:
        
        if new_df['time'].iloc[-2] != df['time'].iloc[-1]: # ถ้าข้อมูลสุดท้ายไม่เท่ากับช้อมูลสุดท้าย
            
            if new_df['time'].iloc[-1] != new_df['time'].iloc[-2]: # เช็คอีกทีก่อนเพิ่มอมูลใหม่
                    
                    test_df = pd.DataFrame(new_df.iloc[-2]).transpose()
                    df = pd.concat([df, test_df])
                    strategy(df)
                    
                    signal = df.iloc[-1, 1]
                    per_sig = df.iloc[-2, 1]
                    
                    # take action
                    if (signal == True) & (pre_sig == False):
                        print('buy at : ' + str(datetime.now()))
                        
                        buy_log.append(datetime.now())
                        
#                         place_order = deri.place_order(
#                             symbol = 'GOU21',
#                             price = 0.0,
#                             volume = 1,
#                             price_type = 'SPECIAL_MARKET',
#                             side = 'LONG',
#                             position = 'AUTO',
#                             pin = '192539'
#                         )
                    elif (signal == False) & (pre_sig == True):
                        print('sell at : ', str(datetime.new()))
            
                        sell_log.append(datetime.now())
                
#                         place_order = deri.place_order(
#                             symbol = 'GOU21',
#                             price = 0.0,
#                             volume = 1,
#                             price_type = 'SPECIAL_MARKET',
#                             side = 'SHORT',
#                             position = 'AUTO',
#                             pin = '192539'
#                         )
    print('update')

sub = mqtt.subscribe_candlestick('GOH22', interval='1m', on_message = get_realtime)
sub.start()

'NoneType' object is not subscriptable


In [13]:
df

,time,open,high,low,close,volume


In [14]:
new_df

,time,open,high,low,close,volumn


In [16]:
sub.stop()

In [26]:
buy_log

[]

In [27]:
sell_log

[]

In [21]:
realtime = investor.RealtimeDataConnection()

def my_message(result, subscriber):
    close = result['data']['close']
    print('close price 1 :', close)

    ## stop this subscription if close price 1 is greater than 100
    if close > 100:
        print("stop subscribe")
        subscriber.stop()

sub = realtime.subscribe_candlestick("AOT",  interval='1m', on_message = my_message)
sub.start()

close price 1 : 65.0
close price 1 : 65.0
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 

close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 65.0
close price 1 : 65.0
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.75
close price 1 : 64.5
close price 1 : 64.5
close price 1 : 64.5
close price 1 : 64.5
close price 1 : 64.5
close price 1 : 6